In [1]:
import os
import faiss
from Libraries import A0_Widgets, A1_Define, A2_MyUtils
from Libraries import B1_ExtractData, B2_MergeData, B3_GetStructures
from Libraries import B4_ChunkMaster, B5_ChunkFlex, B6_ChunkFixed

In [2]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
widgets_list = A0_Widgets.create_name_form()

Dropdown(description='Embedder: ', index=2, layout=Layout(width='90%'), options=('vinai/phobert-base', 'keepit…

Dropdown(description='Searcher: ', index=1, layout=Layout(width='90%'), options=('faiss.IndexHNSWFlat', 'faiss…

Dropdown(description='Reranker: ', layout=Layout(width='90%'), options=('BAAI/bge-reranker-base',), value='BAA…

Dropdown(description='Response: ', layout=Layout(width='90%'), options=('gemini-2.0-flash-exp', 'vinai/PhoGPT-…

Dropdown(description='API Key:', index=4, layout=Layout(width='90%'), options=('AIzaSyDaHS-8h6GJkyVPhoX4svvYeB…

VBox()

In [10]:
config = A1_Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_base = config["chunks_base"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

Embedder: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base
Searcher: faiss.IndexFlatIP
Reranker: BAAI/bge-reranker-base
Responer: gemini-2.0-flash-exp
Data Key: contents
Embe Key: embeddings
File    : HNMU_Regulations
Dcment  : ../Docs/HNMU_Regulations.pdf
Extract : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Extracted.json
Merge   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Merged.json
Struct  : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Struct.json
Chunked : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Chunks.json
Database: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Database.json
Torch   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeds_Merge.pt
Faiss   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeds_Merge.faiss
Mapping : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeds_Merge_mapping.json
Map Data: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeds_Merge_map_data.json
Schema  : ../Data/HNMU_Regulations/Data_HNMU_Regula

In [ ]:
Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None
print(Contents)

### Extract Data

In [ ]:
exceptions_path = "../tests/ex.exceptions.json"
markers_path = "../tests/ex.markers.json"
status_path = "../tests/ex.status.json"

In [ ]:
extracted_data = B1_ExtractData.extractData(dcmt_path, exceptions_path, markers_path, status_path)
A2_MyUtils.write_json(extracted_data, extracted_path, indent=1)

In [ ]:
merged_data = B2_MergeData.mergeLinesToParagraphs(extracted_data)
A2_MyUtils.write_json(merged_data, merged_path, indent=1)

### Get Struct

In [ ]:
analyzer = B3_GetStructures.StructureAnalyzer(merged_path, verbose=True)

In [ ]:
def getMarkers():
    markers = analyzer.extract_markers()
    return markers

In [ ]:
def getStructures(markers):
    structures = analyzer.build_structures(markers)
    print(A2_MyUtils.jsonl_convert(structures))
    return structures

In [ ]:
def getDedup(structures):
    dedup = analyzer.deduplicate(structures)
    print(A2_MyUtils.jsonl_convert(dedup))
    return dedup

In [ ]:
def getTop(dedup):
    top = analyzer.select_top(dedup)
    topext = analyzer.extend_top(top, dedup)
    print(A2_MyUtils.json_convert(topext, pretty=True))
    return topext

In [ ]:
markers = getMarkers()
structures = getStructures(markers)
dedup = getDedup(structures)
topext = getTop(dedup)

In [ ]:
A2_MyUtils.write_json(topext, struct_path, indent=2)

### Chunks

In [ ]:
builder = B4_ChunkMaster.ChunkBuilder(
    struct_path="Data_HNMU_Regulations_Texts_Struct.json",
    merged_path="Data_HNMU_Regulations_Texts_Merged.json",
    output_path="Data_HNMU_Regulations_Chunks.json"
)

builder.build()
chunks = builder.get_chunks()
print(chunks[:2])

In [15]:
import json
from copy import deepcopy
from pathlib import Path
from collections import OrderedDict

# ===== 0) Khai báo đường dẫn =====
# struct_path = Path("Data_HNMU_Regulations_Texts_Struct.json")
# merged_path = Path("Data_HNMU_Regulations_Texts_Merged.json")
chunks_base_path = Path("Data_HNMU_Regulations_Chunks.json")

# ===== 1) Đọc dữ liệu =====
with open(struct_path, "r", encoding="utf-8") as f:
    struct_data = json.load(f)

with open(merged_path, "r", encoding="utf-8") as f:
    merged_data = json.load(f)

# ===== 2) Chuẩn bị cấu trúc =====
struct_spec = struct_data[0]
ordered_fields = list(struct_spec.keys())
last_field = ordered_fields[-1]
level_fields = ordered_fields[:-1]

# Map field -> tập marker mẫu
marker_dict = {}
for fld in ordered_fields:
    vals = struct_spec.get(fld, [])
    marker_dict[fld] = set(vals) if isinstance(vals, list) else set()

def new_temp():
    return {fld: "" for fld in level_fields} | {last_field: []}

def temp_has_data(temp):
    return any(temp[f].strip() for f in level_fields) or bool(temp[last_field])

def reset_deeper(temp, touched_field):
    idx = level_fields.index(touched_field)
    for f in level_fields[idx+1:]:
        temp[f] = ""
    temp[last_field] = []

def has_data_from_level(temp, fld):
    """Kiểm tra từ level fld trở xuống có dữ liệu không"""
    if fld not in level_fields:
        return False
    idx = level_fields.index(fld)
    for f in level_fields[idx:]:
        if temp[f].strip():
            return True
    if temp[last_field]:
        return True
    return False

def with_index(temp, idx):
    """Tạo OrderedDict với Index đứng đầu"""
    od = OrderedDict()
    od["Index"] = idx
    for f in level_fields:
        od[f] = temp[f]
    od[last_field] = temp[last_field]
    return od

# ===== 3) Thuật toán =====
paragraphs = merged_data.get("paragraphs", [])
paragraphs = sorted(paragraphs, key=lambda x: x.get("Paragraph", 0))

chunk_data = []
temp = new_temp()
index_counter = 1  # bộ đếm Index

for p in paragraphs:
    text = p.get("Text", "")
    marker = p.get("MarkerType", None) or "none"

    matched_field = None
    for fld in level_fields:
        if marker in marker_dict.get(fld, set()):
            matched_field = fld
            break

    if matched_field is not None:
        # chỉ append nếu từ matched_field trở xuống có dữ liệu
        if has_data_from_level(temp, matched_field):
            chunk_data.append(with_index(deepcopy(temp), index_counter))
            index_counter += 1

        temp[matched_field] = text
        reset_deeper(temp, matched_field)
    else:
        temp[last_field].append(text)

if temp_has_data(temp):
    chunk_data.append(with_index(deepcopy(temp), index_counter))
    index_counter += 1

# ===== 4) Ghi kết quả =====
with open(chunks_base_path, "w", encoding="utf-8") as f:
    json.dump(chunk_data, f, ensure_ascii=False, indent=2)

print(f"✅ Hoàn thành! Đã tạo {len(chunk_data)} chunks.")
print(f"File kết quả: {chunks_base_path.resolve()}")


✅ Hoàn thành! Đã tạo 160 chunks.
File kết quả: E:\.Data\Documents\Code\4. Python\LLMs\RAG\Main\Data_HNMU_Regulations_Chunks.json
